In [9]:
import pandas as pd
import plotly.graph_objects as go

In [25]:
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Dec24-Feb25_60min.csv')

df['Date'] = pd.to_datetime(df['Date'])

# Calculate Moving Averages
df['MA9'] = df['Close'].rolling(window=9).mean()
df['MA12'] = df['Close'].rolling(window=12).mean()

# Remove rows where MA9 or MA12 are NaN (to avoid gaps in the plot)
df = df.dropna(subset=['MA9', 'MA12'])

# Find intersections where MA9 crosses MA12
intersection_points = []
for i in range(1, len(df)):
    # Check for crossing points (sign change in the difference)
    if (df['MA9'][i] > df['MA12'][i] and df['MA9'][i-1] <= df['MA12'][i-1]) or (df['MA9'][i] < df['MA12'][i] and df['MA9'][i-1] >= df['MA12'][i-1]):
        intersection_points.append((df['Date'][i], df['Close'][i]))

# Create Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add MA9 and MA12 to the chart
fig.add_trace(go.Scatter(x=df['Date'], y=df['MA9'], mode='lines', name='MA9', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['MA12'], mode='lines', name='MA12', line=dict(color='red')))

# Plot intersections as markers
for point in intersection_points:
    fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]], mode='markers', name='Intersection', marker=dict(color='green', size=10)))

# Update layout to not leave space for missing dates
fig.update_layout(title='Candlestick with MA9 and MA12 Intersections',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show interactive plot
fig.show()

KeyError: 1